# Projet MRO - COP

Dans ce NoteBook, on s'applique à trouver des solutions pour notre modélisation sur le probèle d'attribution de fréquences.

## Bibliothèques

In [1]:
import json
from itertools import product
from numpy import abs


## Initialisation de la modélisation

### Chargement des données

In [2]:
data='test.json'
max_bound = 10000
nb_fct_cout = 0
fqc_max = 728 +1

### Chargemment des données et tableau utiles à l'écriture des contraintes

In [3]:
with open(data, 'r') as f:
    f = json.load(f)
    stations=f['stations']
    liaisons = f['liaisons']
    interferences = f['interferences']
    regions = f['regions']
    nb_stations = len(stations)

taille_domaine = (2 * nb_stations-1) * f'{fqc_max} '+ f'{fqc_max}\n'


### Écriture des contraintes de notre modélisation

In [5]:
contraintes_fqc = f''
for station in stations:
    fqc_e = station['emetteur']
    fqc_r = station['recepteur']
    nb_tuple_e = 0
    nb_tuple_r = 0
    contraintes_e = f''
    contraintes_r = f''
    for fqc in fqc_e:
        contraintes_e += f'{fqc} 0\n' 
        nb_tuple_e+=1
    for fqc in fqc_r:
        contraintes_r += f'{fqc} 0\n' 
        nb_tuple_r+=1

    contraintes_fqc += f'1 {station["num"]} {max_bound} {nb_tuple_e}\n' + contraintes_e
    contraintes_fqc += f'1 {station["num"]+nb_stations} {max_bound} {nb_tuple_r}\n' + contraintes_r
    nb_fct_cout+=2

In [7]:
delta = stations[0]["delta"]
contraintes_delta = f''
for station in stations:
    contrainte_delta = f''
    fqc_e = station['emetteur']
    fqc_r = station['recepteur']
    fqc_possible = list(product(fqc_e, fqc_r))
    nb_tuple_delta = 0
    for item in fqc_possible:
        if abs(item[0]-item[1]) == station["delta"]:
            contrainte_delta += f'{item[0]} {item[1]} 0\n'
            contrainte_delta += f'{item[1]} {item[0]} 0\n'
            nb_tuple_delta +=2
    contraintes_delta += f'2 {station["num"]} {station["num"] + nb_stations} {max_bound} {nb_tuple_delta}\n' + contrainte_delta
    nb_fct_cout+=1



Contraintes par intension "disj". Elles permettent de vérifier que $x \geq y + csty$ ou $y \geq x + cstx$, ce qui nous permet d'inclure la disjonction induite par la valeur absolue pour vérifer que l'on ait supérieur à $\Delta$. $cstx$ et $csty$ sont précisés après `disj`, le dernier argument étant la pénalité que l'on veut mettre à notre fonction de coût (ici contrainte molle de coût $\Delta$ si violée)

In [9]:
contraintes_Delta = f''
for interference in interferences:
    contrainte_Delta_e_x_e_y = f'2 {interference["x"]} {interference["y"]} -1 disj {interference["Delta"]} {interference["Delta"]} {interference["Delta"]}\n' 
    contrainte_Delta_e_x_r_y = f'2 {interference["x"]} {interference["y"]+nb_stations} -1 disj {interference["Delta"]} {interference["Delta"]} {interference["Delta"]}\n'
    contrainte_Delta_r_x_e_y = f'2 {interference["x"]+nb_stations} {interference["y"]} -1 disj {interference["Delta"]} {interference["Delta"]} {interference["Delta"]}\n'
    contrainte_Delta_r_x_r_y = f'2 {interference["x"]+nb_stations} {interference["y"]+nb_stations} -1 disj {interference["Delta"]} {interference["Delta"]} {interference["Delta"]}\n'

    contraintes_Delta+= contrainte_Delta_e_x_e_y
    contraintes_Delta+= contrainte_Delta_e_x_r_y
    contraintes_Delta+= contrainte_Delta_r_x_e_y
    contraintes_Delta+= contrainte_Delta_r_x_r_y
    nb_fct_cout+=4

Contraintes par intension `=` (voir doc Toulbar2)

In [10]:
contraintes_liaisons = f''
for liaison in liaisons:
    contraintes_liaisons += f'2 {liaison["x"]+nb_stations} {liaison["y"]} -1 = 0 0\n'
    contraintes_liaisons += f'2 {liaison["x"]} {liaison["y"]+nb_stations} -1 = 0 0\n'
    nb_fct_cout+=2

## Écriture du fichier WCSP

In [11]:
fichier = open("allocation_fqc.wcsp", 'w')
fichier.write(f'frequenceWCSP {2*nb_stations} {fqc_max} {nb_fct_cout} {max_bound}\n' + taille_domaine + contraintes_Delta + contraintes_liaisons+ contraintes_delta + contraintes_fqc)
fichier.close()